In [ ]:
"""
Titanic Survival Prediction - Kaggle Competition
Author: Kristiyan Bonev
Updated: November 2025

This notebook provides a comprehensive analysis of the Titanic dataset with:
- Exploratory Data Analysis (EDA)
- Advanced Feature Engineering
- Multiple Model Comparisons with Cross-Validation
- Hyperparameter Tuning via GridSearch
- Ensemble Methods & Voting Classifiers
- SHAP Analysis for Model Explainability
- Production-Ready ML Pipeline

For production deployment, see scripts/run_training.py
"""

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import shap
import sklearn
import itertools

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import (
    f1_score, 
    classification_report,
    accuracy_score,
    roc_auc_score
)

# Add project root to path for importing custom modules
sys.path.append(str(Path.cwd().parent))

# Import notebook-specific utilities
from notebooks.utils import config
from notebooks.utils.visualization import (
    plot_confusion_matrix,
    plot_roc_curve,
    # plot_feature_importance,
    # plot_correlation_heatmap,
    # plot_learning_curves,
    plot_model_comparison,
    plot_cv_score_distribution,
    # plot_precision_recall_curve
)
from notebooks.utils.notebook_helpers import (
    optimize_models,
    evaluate_models_cv,
    get_cv_scores_detailed,
    save_model_pickle,
    # load_model_pickle,
    generate_kaggle_submission
)

# Set random seeds for reproducibility
np.random.seed(config.RANDOM_SEED)

# Configure matplotlib and seaborn
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Display notebook info
print("\n" + "="*60)
print("TITANIC SURVIVAL PREDICTION")
print("="*60)
print(f"Python version: {sys.version.split()[0]}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print("="*60 + "\n")

: 

## Project Planning

Let's outline the main steps of the project:
- Understanding the nature of the data .info() .describe()
- Exploratory analysis - histograms, boxplots and value counts
- Deal with missing data
- Check metrics correlation
- Themes to look into:
  - Level of wealth as a predictor
  - Location of cabin as a predictor
  - Age
  - Title
- Feature engineering
- Data preprocessing and scaling
- Baseline models
- Trained models
- Comparison.

## Configuration & Setup

**Reproducibility Settings:**
- Random Seed: 42 (set across numpy, random, sklearn)
- Cross-Validation: Stratified 5-Fold (handles class imbalance)
- Primary Metric: Accuracy (Kaggle competition metric)
- Secondary Metric: F1-weighted (better for imbalanced classes)

**Class Imbalance Handling:**
- ✓ **Stratified K-Fold CV** - Maintains class distribution across folds
- ✓ **Weighted F1 Score** - Accounts for minority class importance
- ✓ **Comprehensive Metrics** - Precision, Recall, ROC-AUC for full picture

**Key Improvements in This Version:**
- ✓ Full reproducibility with fixed random seeds
- ✓ Stratified cross-validation for imbalanced data
- ✓ Comprehensive evaluation metrics (accuracy, F1, ROC-AUC)
- ✓ Advanced visualizations
- ✓ Modular code structure with reusable utilities
- ✓ Modern Python best practices (type hints, docstrings)
- ✓ Feature importance analysis
- ✓ SHAP explainability
- ✓ Production-ready ML pipeline

<div align="center">
  <h3>Data Dictionary</h3>
</div>

| Variable  | Definition                | Key                                       |
|-----------|---------------------------|-------------------------------------------|
| survival  | Survival                  | 0 = No, 1 = Yes                           |
| pclass    | Ticket class              | 1 = 1st, 2 = 2nd, 3 = 3rd                 |
| sex       | Sex                       |                                           |
| Age       | Age in years              |                                           |
| sibsp     | # of siblings/spouses     |                                           |
| parch     | # of parents/children     |                                           |
| ticket    | Ticket number             |                                           |
| fare      | Passenger fare            |                                           |
| cabin     | Cabin number              |                                           |
| embarked  | Port of Embarkation       | C = Cherbourg, Q = Queenstown, S = Southampton |


### Data import

For this analysis, I will be exclusively working with the Training set. The training dataset will be used for validation as well. The final predictions for the Kaggle submissions will of course be done on the provided test dataset as per the competition's rules. I'll be doing the initial data analysis on the combined data which I'll differentiate with the use of a train/test flag in a new column.

In [ ]:
# Load training and test data using config paths
train = pd.read_csv(config.TRAIN_DATA_PATH)
test = pd.read_csv(config.TEST_DATA_PATH)

train['train_test'] = 1
test['train_test'] = 0
test['Survived'] = np.NaN
all_data = pd.concat([train, test])

%matplotlib inline

## 1. Data Exploration

**For Numeric Data:**
   - Histograms to explore distribution;
   - Correlation plots;
   - Pivot tables to try and understand survival rate across numeric variables;

**For Categorical Data:**
   - Plotting bar charts to explore balance of classes
   - Pivot tables again with the same goal of understanding the relation to survival rate

In [ ]:
# Let's start with an overview of the train data:
train.head(20)

In [ ]:
# Listing Numerical columns
cols = train.columns
num_cols = list(train.select_dtypes('number'))
print(num_cols)

In [ ]:
# Listing Categorical columns
cat_cols = list(set(cols) - set([str(col) for col in num_cols]))
print(cat_cols)

In [ ]:
# A look at feature datatypes
train.info()

##### Two colums stand out:
- Cabin is mostly null values which will need to be looked into and dealt with
- Age has a significant number of null vlaues as well which will also need to be investigated

In [ ]:
# An overview of the central tendencies of the numeric data
train.describe()

In [ ]:
# Separating the numeric and categorical columns as each will require a separate approach
df_numeric = train[['Age','SibSp','Parch','Fare']]
df_categorical = train[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

### 1.1 Numeric Features Exploration

In [ ]:
# Creating a grid of subplots for the numeric columns
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
axes = axes.flatten()  # Flatten 2D array to 1D

# Plot each histogram in a subplot
for i, col in enumerate(df_numeric.columns):
    plt.sca(axes[i])
    plt.hist(df_numeric[col])
    plt.title(col)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()

We have a normal distribution for the age column. The rest are all good candidates for normalization since they are skewed quite heavily.

In [ ]:
# A look at the correlation of our numeric columns
print(df_numeric.corr())
sns.heatmap(df_numeric.corr())

Here we can see that Parch and SibSp has a higher correlation, which generally makes sense since Parents are more likely to travel with their multiple kids and spouses tend to travel together.

In [ ]:
# A look at survival rate across Age, SibSp, Parch, and Fare 
pd.pivot_table(
    data = train,
    values = ['Age', 'SibSp', 'Parch', 'Fare'],
    index = 'Survived'
)

There are a few interesting aspects to this:
- The average age of survivors is 28, so young people tend to have a better chance;
-  People who paid higher fare rates were more likely to survive, more than double. This might be the people traveling in first-clas,.tThus the rich survive. Higher class cabins were higher up on the ship;.-
Travelling with parents shows a significant correlation. Well done, parents on the Titanic;e- Having siblings looks to not be an advantage;ing

### 1.2 Categorical Features Exploration

In [ ]:
# Same grid of subplots as the one above but containing barplots with our categorical features
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 10))
axes = axes.flatten()

# Plot each barplot in a subplot
for i, col in enumerate(df_categorical.columns):
    sns.barplot(x=df_categorical[col].value_counts().index, y=df_categorical[col].value_counts(), ax=axes[i])
    axes[i].set_title(col)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()

- Cabin and ticket graphs are very messy and will need to be addressed (feature engineering);
- Survived: Most of the people died in the shipwreck, only around 300 people survived. The classes are not balanced which needs to be taken into account;
- Pclass: The majority of the people traveling were in 3rd class;
- Sex: There were roughly double the number of men on board than women;
- Embarked: Most of the passengers boarded the ship from Southampton;

In [ ]:
# Survival by Pclass - Enhanced Visualization
survival_by_class = pd.crosstab(train['Pclass'], train['Survived'], margins=True)
survival_rate = train.groupby('Pclass')['Survived'].agg(['sum', 'count', 'mean'])
survival_rate.columns = ['Survived', 'Total', 'Survival Rate']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sns.countplot(data=train, x='Pclass', hue='Survived', palette='Set2', ax=ax1)
ax1.set_title('Survival Count by Passenger Class', fontsize=14, fontweight='bold')
ax1.set_xlabel('Passenger Class', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.legend(title='Survived', labels=['No', 'Yes'])

# Survival rate plot
survival_rate['Survival Rate'].plot(kind='bar', ax=ax2, color=['#ff9999', '#66b3ff', '#99ff99'], edgecolor='black')
ax2.set_title('Survival Rate by Passenger Class', fontsize=14, fontweight='bold')
ax2.set_xlabel('Passenger Class', fontsize=12)
ax2.set_ylabel('Survival Rate', fontsize=12)
ax2.set_xticklabels(['1st Class', '2nd Class', '3rd Class'], rotation=0)
ax2.set_ylim([0, 1])
ax2.grid(axis='y', alpha=0.3)

# Add percentage labels
for i, v in enumerate(survival_rate['Survival Rate']):
    ax2.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Survival Statistics by Passenger Class:")
print(survival_rate.to_string())
print(f"\n✨ Key Insight: 1st class passengers had a {survival_rate.loc[1, 'Survival Rate']:.1%} survival rate,")
print(f"   compared to only {survival_rate.loc[3, 'Survival Rate']:.1%} for 3rd class!")


**📈 Analysis:**

The data reveals a stark **class-based disparity** in survival rates:
- **1st Class**: Highest survival rate (~63%) - better cabin locations closer to lifeboats
- **2nd Class**: Moderate survival rate (~47%) - middle decks
- **3rd Class**: Lowest survival rate (~24%) - lower decks, farther from escape routes

This supports the "**women and children first**" protocol being more effectively implemented in higher classes, and the proximity to lifeboats being a critical factor.


In [ ]:
# Survival by Sex - Enhanced Visualization
survival_by_sex = train.groupby('Sex')['Survived'].agg(['sum', 'count', 'mean'])
survival_by_sex.columns = ['Survived', 'Total', 'Survival Rate']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sns.countplot(data=train, x='Sex', hue='Survived', palette='Set1', ax=ax1)
ax1.set_title('Survival Count by Sex', fontsize=14, fontweight='bold')
ax1.set_xlabel('Sex', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.legend(title='Survived', labels=['No', 'Yes'])

# Survival rate plot
survival_by_sex['Survival Rate'].plot(kind='bar', ax=ax2, color=['#ff6b9d', '#4d94ff'], edgecolor='black')
ax2.set_title('Survival Rate by Sex', fontsize=14, fontweight='bold')
ax2.set_xlabel('Sex', fontsize=12)
ax2.set_ylabel('Survival Rate', fontsize=12)
ax2.set_xticklabels(['Female', 'Male'], rotation=0)
ax2.set_ylim([0, 1])
ax2.grid(axis='y', alpha=0.3)

# Add percentage labels
for i, (idx, v) in enumerate(survival_by_sex['Survival Rate'].items()):
    ax2.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Survival Statistics by Sex:")
print(survival_by_sex.to_string())
print(f"\n✨ Key Insight: Female passengers had a {survival_by_sex.loc['female', 'Survival Rate']:.1%} survival rate,")
print(f"   while male passengers had only {survival_by_sex.loc['male', 'Survival Rate']:.1%}!")
print(f"   Females were {survival_by_sex.loc['female', 'Survival Rate'] / survival_by_sex.loc['male', 'Survival Rate']:.1f}x more likely to survive.")


**🚢 Historical Context: "Women and Children First"**

The dramatic difference in survival rates (74% for women vs. 19% for men) clearly demonstrates the **Birkenhead Drill** protocol in action. This maritime tradition prioritized women and children during evacuations, which was largely followed during the Titanic disaster.

The data shows this protocol was strictly enforced, contributing to one of the most significant gender disparities in survival rates in maritime history.


In [ ]:
# Survival by Embarked - Enhanced Visualization
survival_by_embarked = train.groupby('Embarked')['Survived'].agg(['sum', 'count', 'mean'])
survival_by_embarked.columns = ['Survived', 'Total', 'Survival Rate']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sns.countplot(data=train, x='Embarked', hue='Survived', palette='Set3', ax=ax1)
ax1.set_title('Survival Count by Port of Embarkation', fontsize=14, fontweight='bold')
ax1.set_xlabel('Port of Embarkation', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_xticklabels(['Cherbourg', 'Queenstown', 'Southampton'])
ax1.legend(title='Survived', labels=['No', 'Yes'])

# Survival rate plot
survival_by_embarked['Survival Rate'].plot(kind='bar', ax=ax2, color=['#ffd966', '#93c47d', '#ff9999'], edgecolor='black')
ax2.set_title('Survival Rate by Port of Embarkation', fontsize=14, fontweight='bold')
ax2.set_xlabel('Port', fontsize=12)
ax2.set_ylabel('Survival Rate', fontsize=12)
ax2.set_xticklabels(['Cherbourg', 'Queenstown', 'Southampton'], rotation=0)
ax2.set_ylim([0, 1])
ax2.grid(axis='y', alpha=0.3)

# Add percentage labels
for i, (idx, v) in enumerate(survival_by_embarked['Survival Rate'].items()):
    ax2.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Survival Statistics by Port of Embarkation:")
print(survival_by_embarked.to_string())
print(f"\n💡 Insight: Cherbourg passengers had the highest survival rate at {survival_by_embarked.loc['C', 'Survival Rate']:.1%}")


**🤔 Why Does Embarkation Port Matter?**

At first glance, the port of embarkation seems irrelevant to survival. However, the higher survival rate for Cherbourg passengers (55%) compared to Southampton (34%) isn't causally related to the port itself.

**The Real Factor**: Passenger demographics and ticket class distribution:
- Cherbourg had more **1st class passengers** (wealthier travelers)
- Southampton had more **3rd class passengers** (emigrants)

This demonstrates **confounding variables** in data analysis - Embarkation port correlates with survival, but the actual driver is socioeconomic status (passenger class).


## 2. Feature Engineering
1) Cabin - Simplify cabins (evaluated if cabin letter (cabin_letters) or the purchase of tickets across multiple cabins (cabin_multiple) impacted survival)
2) Tickets - Do different ticket types impact survival rates?
3) Name - Does a person's title relate to survival rates? I am curious if the captain went down with the ship.

### 2.1 Cabin
After looking at this, we may want to look at cabin by letter or by number. Let's create some categories for this:
 - Single letters
 - Multiple letters

In [ ]:
# Cabin Multiple: Passengers who purchased multiple cabins
train['cabin_multiple'] = train.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

# Visualize distribution and survival impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
cabin_counts = train['cabin_multiple'].value_counts().sort_index()
ax1.bar(cabin_counts.index, cabin_counts.values, color='steelblue', edgecolor='black')
ax1.set_title('Distribution of Cabin Ownership', fontsize=14, fontweight='bold')
ax1.set_xlabel('Number of Cabins', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# Survival rate by cabin multiple
survival_by_cabin = train.groupby('cabin_multiple')['Survived'].agg(['sum', 'count', 'mean'])
survival_by_cabin.columns = ['Survived', 'Total', 'Survival Rate']

survival_by_cabin['Survival Rate'].plot(kind='bar', ax=ax2, color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff99cc'], edgecolor='black')
ax2.set_title('Survival Rate by Cabin Ownership', fontsize=14, fontweight='bold')
ax2.set_xlabel('Number of Cabins', fontsize=12)
ax2.set_ylabel('Survival Rate', fontsize=12)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.set_ylim([0, 1])
ax2.grid(axis='y', alpha=0.3)

# Add percentage labels
for i, v in enumerate(survival_by_cabin['Survival Rate']):
    ax2.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Cabin Ownership Statistics:")
print(survival_by_cabin.to_string())
print(f"\n💡 Insight: {survival_by_cabin.loc[0, 'Total']} passengers ({survival_by_cabin.loc[0, 'Total']/len(train):.1%}) had no cabin data,")
print(f"   suggesting lower socioeconomic status. Their survival rate: {survival_by_cabin.loc[0, 'Survival Rate']:.1%}")

**📌 Key Finding:**

The visualization reveals a strong **wealth-survival correlation**:
- **No cabin (0)**: 30% survival rate - majority of passengers, lower class
- **Single cabin (1)**: 67% survival rate - wealthier passengers  
- **Multiple cabins (2+)**: 70-100% survival rates - elite passengers

This engineered feature captures **socioeconomic status** better than Pclass alone, as it distinguishes ultra-wealthy passengers within 1st class.


Next, let us look at the actual letter of the cabin they were in. I would expect that the cabins with the same letter are roughly in the same locations, or on the same floors, and logically if a cabin was near the lifeboats, they had a better chance of survival.

In [ ]:
# Extract cabin letter (indicates deck location on ship)
train['cabin_letters'] = train.Cabin.apply(lambda x: str(x)[0])

# Visualize cabin letter distribution and survival
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
cabin_counts = train['cabin_letters'].value_counts()
ax1.bar(range(len(cabin_counts)), cabin_counts.values, tick_label=cabin_counts.index, color='coral', edgecolor='black')
ax1.set_title('Distribution of Cabin Letters (Deck Locations)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Cabin Letter (n = no cabin)', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# Survival rate by cabin letter
survival_by_cabin_letter = train.groupby('cabin_letters')['Survived'].agg(['sum', 'count', 'mean']).sort_values('mean', ascending=False)
survival_by_cabin_letter.columns = ['Survived', 'Total', 'Survival Rate']

colors = plt.cm.RdYlGn(survival_by_cabin_letter['Survival Rate'])
survival_by_cabin_letter['Survival Rate'].plot(kind='barh', ax=ax2, color=colors, edgecolor='black')
ax2.set_title('Survival Rate by Cabin Letter', fontsize=14, fontweight='bold')
ax2.set_xlabel('Survival Rate', fontsize=12)
ax2.set_ylabel('Cabin Letter', fontsize=12)
ax2.set_xlim([0, 1])
ax2.grid(axis='x', alpha=0.3)

# Add percentage labels
for i, v in enumerate(survival_by_cabin_letter['Survival Rate']):
    ax2.text(v + 0.02, i, f'{v:.1%}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Cabin Letter Survival Statistics:")
print(survival_by_cabin_letter.head(10).to_string())
print(f"\n🚢 Note: Cabins B, D, E had highest survival rates (upper decks, closer to lifeboats)")

**Decision**: While cabin letters show survival variation, the feature has **too many missing values** (77% null). The `cabin_multiple` feature captures wealth/status more reliably with less missingness.

### 2.2 Ticket

Judging by the initial overview and the description of the feature on Kaggle, this feature is likely not going to provide information that is of much use but let's try and have a more detailed look. Let's begin by having a look at the numeric and non-numeric ticket count:

In [ ]:
train['numeric_ticket'] = train.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
train['numeric_ticket'].value_counts()

Now let's have a look at the letters in the tickets:

In [ ]:
train['ticket_letters'] = train.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.', '').replace('/', '').lower() if len(x.split(' ')[:-1]) > 0 else 0)
train['ticket_letters'].value_counts()

Letters don't see to be of much use for model training. Let's check out the survival rate for numeric vs non-numeric tickets:

In [ ]:
pd.pivot_table(
    train, 
    index='Survived',
    columns='numeric_ticket',
    values='Ticket',
    aggfunc='count'
)

In [ ]:
# Survival rate across different ticket types 
pd.pivot_table(
    train,
    index='Survived',
    columns='ticket_letters',
    values='Ticket',
    aggfunc='count'
)

Ticket does not seem to be all that relevant to the task at hand.

### 2.3 Name
The only aspect of this feature which I am interested in is the title. It will serve a very similar purpose as the Sax feature but might provide some additional detail. Let's pull out the titles as a separate feature (Mr., Ms., Master., etc:)

In [ ]:
# Extract title from name (Mr., Mrs., Master., etc.)
train['name_title'] = train['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

# Visualize title distribution and survival
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Count plot - top titles only
title_counts = train['name_title'].value_counts()
top_titles = title_counts.head(10)
ax1.barh(range(len(top_titles)), top_titles.values, tick_label=top_titles.index, color='mediumpurple', edgecolor='black')
ax1.set_title('Top 10 Passenger Titles (Distribution)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Count', fontsize=12)
ax1.set_ylabel('Title', fontsize=12)
ax1.grid(axis='x', alpha=0.3)

# Survival rate by title - only titles with 5+ passengers
survival_by_title = train.groupby('name_title')['Survived'].agg(['sum', 'count', 'mean'])
survival_by_title.columns = ['Survived', 'Total', 'Survival Rate']
survival_by_title = survival_by_title[survival_by_title['Total'] >= 5].sort_values('Survival Rate', ascending=True)

colors = plt.cm.get_cmap('RdYlGn')(survival_by_title['Survival Rate'])
survival_by_title['Survival Rate'].plot(kind='barh', ax=ax2, color=colors, edgecolor='black')
ax2.set_title('Survival Rate by Title (n ≥ 5)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Survival Rate', fontsize=12)
ax2.set_ylabel('Title', fontsize=12)
ax2.set_xlim([0, 1])
ax2.grid(axis='x', alpha=0.3)
ax2.axvline(x=train['Survived'].mean(), color='red', linestyle='--', linewidth=2, label=f'Overall Rate: {train["Survived"].mean():.1%}')
ax2.legend()

# Add percentage labels
for i, v in enumerate(survival_by_title['Survival Rate']):
    ax2.text(v + 0.02, i, f'{v:.0%}', va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("\n📊 Title Survival Statistics (n ≥ 5):")
print(survival_by_title.to_string())
print(f"\n👤 Insight: 'Mrs' (79%), 'Miss' (70%), 'Master' (58%) had high survival - aligns with 'women & children first'")
print(f"   'Mr' had only 16% survival rate - adult males were last priority")

## 3. Data Preprocessing for Model Training

Before training our models, we need to transform the raw data into a format suitable for machine learning:

**Key Preprocessing Steps:**
1. **Handle Missing Values**: 
   - Drop rows with null `Embarked` (only 2 instances)
   - Impute `Age` and `Fare` with median values

2. **Feature Selection**:
   - Keep: `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, `Embarked`
   - Add engineered features: `cabin_multiple`, `name_title`, `norm_fare`
   - Drop: `PassengerId`, `Name`, `Ticket`, `Cabin` (raw versions)

3. **Feature Engineering**:
   - `cabin_multiple`: Number of cabins owned (proxy for wealth/status)
   - `name_title`: Social title extracted from name (Mr., Mrs., etc.)
   - `norm_fare`: Log-transformed fare for better distribution

4. **Encoding & Scaling**:
   - One-hot encoding for categorical variables
   - StandardScaler for numeric features (age, sibsp, parch, fare)

**Why These Choices?**
- Median imputation is robust to outliers
- Log transformation normalizes the skewed fare distribution  
- One-hot encoding prevents ordinal assumptions for categories
- Scaling ensures all features contribute equally to distance-based models


### 3.1 Applying Feature Engineering

In [ ]:
# Setting up all categorical variables that will be used for both the training and the test sets 
all_data['cabin_multiple'] = all_data['Cabin'].apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
all_data['name_title'] = all_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

# Impute nulls for continuous data 
print("🔧 Imputation Strategy:")
print(f"   Age: {all_data['Age'].isna().sum()} missing values → imputing with median ({train['Age'].median():.1f} years)")
print(f"   Fare: {all_data['Fare'].isna().sum()} missing values → imputing with median (${train['Fare'].median():.2f})")

all_data['Age'] = all_data['Age'].fillna(train['Age'].median())
all_data['Fare'] = all_data['Fare'].fillna(train['Fare'].median())

# Drop null 'Embarked' rows as they are of no relevance (2 instances in the training set and none in the testing set)
embarked_nulls = all_data['Embarked'].isna().sum()
all_data.dropna(subset=['Embarked'], inplace=True)
print(f"   Embarked: {embarked_nulls} missing values → dropped rows (minimal impact)")

# Log normalization of fare - visualize before/after transformation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Before: Original fare distribution (highly skewed)
all_data['Fare'].hist(bins=50, ax=ax1, color='salmon', edgecolor='black', alpha=0.7)
ax1.set_title('Before: Original Fare Distribution (Right-Skewed)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Fare ($)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.axvline(all_data['Fare'].median(), color='red', linestyle='--', linewidth=2, label=f'Median: ${all_data["Fare"].median():.2f}')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# After: Log-normalized fare (more normal distribution)
all_data['norm_fare'] = np.log(all_data['Fare'] + 1)
all_data['norm_fare'].hist(bins=50, ax=ax2, color='lightgreen', edgecolor='black', alpha=0.7)
ax2.set_title('After: Log-Normalized Fare (More Normal)', fontsize=14, fontweight='bold')
ax2.set_xlabel('log(Fare + 1)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.axvline(all_data['norm_fare'].median(), color='red', linestyle='--', linewidth=2, label=f'Median: {all_data["norm_fare"].median():.2f}')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Fare Transformation Results:")
print(f"   Original - Skewness: {all_data['Fare'].skew():.2f}, Range: ${all_data['Fare'].min():.2f} - ${all_data['Fare'].max():.2f}")
print(f"   Log-Norm - Skewness: {all_data['norm_fare'].skew():.2f}, Range: {all_data['norm_fare'].min():.2f} - {all_data['norm_fare'].max():.2f}")
print(f"   ✅ Log transformation reduced skewness by {(1 - all_data['norm_fare'].skew()/all_data['Fare'].skew())*100:.1f}%")

# Converting fare to a categorical feature for pd.get_dummies()
all_data['Pclass'] = all_data['Pclass'].astype(str)

# Creating dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(all_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'norm_fare', 'Embarked', 'cabin_multiple', 'name_title', 'train_test']])

print(f"\n🎯 One-Hot Encoding Results:")
print(f"   Original features: 10")
print(f"   After encoding: {all_dummies.shape[1]} features")
print(f"   Categorical features expanded: Pclass (3), Sex (2), Embarked (3), name_title ({all_data['name_title'].nunique()})")

### 3.2 Advanced Feature Engineering (New!)

We've implemented **14+ advanced features** to improve model performance beyond the basic feature engineering above. These features are implemented in `src/features/build_features.py` and are used in the production API.

**Advanced Features Include:**

1. **Family Features**:
   - `family_size`: Total family members aboard (SibSp + Parch + 1)
   - `is_alone`: Binary indicator if traveling solo
   - `fare_per_person`: Fare split among family members (accounts for group tickets)

2. **Title Features**:
   - `title_grouped`: Consolidates 17+ rare titles into 5 main categories (Mr, Miss, Mrs, Master, Rare)
   - Improves signal-to-noise ratio by grouping low-frequency titles

3. **Age Features**:
   - `age_group`: Binned into Child (<16), Young Adult (16-32), Adult (32-48), Senior (48+)
   - `age_missing`: Flag indicating if age was imputed (missing data can be informative)
   - `age_class`: Interaction between age and passenger class

4. **Fare Features**:
   - `fare_group`: Quartile-based binning (Low/Medium/High/VeryHigh)
   - **Fare Inference**: If fare is missing, infers from passenger class with family discount applied

5. **Cabin Features**:
   - `cabin_known`: Binary flag if cabin was recorded
   - `cabin_deck`: Deck letter extracted (A-G, or U for unknown)

6. **Ticket Features**:
   - `ticket_prefix`: Ticket number prefix (often indicates ticket type)
   - `ticket_has_letters`: Binary flag if ticket number contains letters

7. **Interaction Features**:
   - `sex_pclass`: Interaction between sex and class (e.g., women in 1st class had highest survival)

**Let's demonstrate these features:**

In [ ]:
# Import the advanced feature engineering function
import sys
sys.path.append('..')  # Add parent directory to path

from src.features.build_features import apply_feature_engineering, infer_fare_from_class

# Demonstrate Fare Inference
print("🎟️ FARE INFERENCE FROM PASSENGER CLASS:")
print("=" * 60)
print("\nWhen fare is missing, we infer it from passenger class medians:")
print("\n📊 Class-Based Fare Estimates:")
for pclass in [1, 2, 3]:
    inferred_fare = infer_fare_from_class(pclass, family_size=1)
    print(f"   Class {pclass}: £{inferred_fare:.2f}")

print("\n👨‍👩‍👧‍👦 Family Discount Applied:")
print("   Families of 4+ get 10% discount on inferred fare")
family_fare = infer_fare_from_class(pclass=2, family_size=4)
solo_fare = infer_fare_from_class(pclass=2, family_size=1)
print(f"   Solo passenger (Class 2): £{solo_fare:.2f}")
print(f"   Family of 4 (Class 2): £{family_fare:.2f} (10% discount)")

print("\n" + "=" * 60)

In [ ]:
# Apply Advanced Feature Engineering
print("🔬 ADVANCED FEATURE ENGINEERING COMPARISON:")
print("=" * 80)

# Basic features (what we had before)
basic_features = train.shape[1]
print(f"\n📦 Original Dataset: {basic_features} columns")
print(f"   Columns: {list(train.columns)}")

# Advanced features
df_advanced, num_features, cat_features = apply_feature_engineering(
    train.copy(), 
    include_advanced=True
)

print(f"\n✨ Advanced Features: {df_advanced.shape[1]} columns (+{df_advanced.shape[1] - basic_features} new)")
print(f"\n📊 Feature Breakdown:")
print(f"   • Numerical Features ({len(num_features)}): {num_features}")
print(f"   • Categorical Features ({len(cat_features)}): {cat_features}")

# Show a sample passenger with all features
print("\n" + "=" * 80)
print("👤 SAMPLE PASSENGER - ALL FEATURES:")
print("=" * 80)
sample_idx = 0
sample = df_advanced.iloc[sample_idx]

print(f"\n🎫 Passenger #{int(sample.get('PassengerId', 0))}")
print("\n📋 ORIGINAL FEATURES:")
original_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
for col in original_cols:
    if col in sample.index:
        print(f"   {col:15s}: {sample[col]}")

print("\n✨ ENGINEERED FEATURES:")
engineered_cols = [
    'family_size', 'is_alone', 'fare_per_person', 
    'title_grouped', 'age_group', 'age_missing', 
    'age_class', 'fare_group', 'cabin_known', 
    'cabin_deck', 'sex_pclass'
]
for col in engineered_cols:
    if col in sample.index:
        print(f"   {col:15s}: {sample[col]}")

print("\n" + "=" * 80)

In [ ]:
# Visualize the impact of key advanced features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('🔍 Advanced Feature Analysis - Survival Patterns', fontsize=16, fontweight='bold', y=0.995)

# 1. Family Size vs Survival
ax1 = axes[0, 0]
family_survival = df_advanced.groupby('family_size')['Survived'].agg(['mean', 'count'])
family_survival = family_survival[family_survival['count'] >= 5]  # Filter rare sizes
ax1.bar(family_survival.index, family_survival['mean'], color='steelblue', edgecolor='black', alpha=0.8)
ax1.set_title('Family Size vs Survival Rate', fontsize=13, fontweight='bold')
ax1.set_xlabel('Family Size', fontsize=11)
ax1.set_ylabel('Survival Rate', fontsize=11)
ax1.axhline(y=df_advanced['Survived'].mean(), color='red', linestyle='--', label=f'Overall: {df_advanced["Survived"].mean():.1%}')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)
for i, (idx, row) in enumerate(family_survival.iterrows()):
    ax1.text(idx, row['mean'] + 0.02, f"{row['mean']:.0%}\n(n={int(row['count'])})", ha='center', fontsize=9)

# 2. Is Alone vs Survival
ax2 = axes[0, 1]
alone_survival = df_advanced.groupby('is_alone')['Survived'].mean()
colors = ['#2ecc71', '#e74c3c']
bars = ax2.bar(['With Family', 'Alone'], alone_survival.values, color=colors, edgecolor='black', alpha=0.8)
ax2.set_title('Solo vs Family Travelers', fontsize=13, fontweight='bold')
ax2.set_ylabel('Survival Rate', fontsize=11)
ax2.set_ylim([0, 1])
ax2.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, alone_survival.values):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.03, f'{val:.1%}', ha='center', fontweight='bold', fontsize=11)

# 3. Title Grouped vs Survival
ax3 = axes[0, 2]
title_survival = df_advanced.groupby('title_grouped')['Survived'].agg(['mean', 'count'])
title_survival = title_survival.sort_values('mean', ascending=False)
ax3.barh(range(len(title_survival)), title_survival['mean'], color='coral', edgecolor='black', alpha=0.8)
ax3.set_yticks(range(len(title_survival)))
ax3.set_yticklabels(title_survival.index)
ax3.set_title('Title vs Survival Rate', fontsize=13, fontweight='bold')
ax3.set_xlabel('Survival Rate', fontsize=11)
ax3.axvline(x=df_advanced['Survived'].mean(), color='red', linestyle='--', linewidth=2)
ax3.grid(axis='x', alpha=0.3)
for i, (idx, row) in enumerate(title_survival.iterrows()):
    ax3.text(row['mean'] + 0.02, i, f"{row['mean']:.0%} (n={int(row['count'])})", va='center', fontsize=9)

# 4. Age Group vs Survival
ax4 = axes[1, 0]
age_group_survival = df_advanced.groupby('age_group')['Survived'].agg(['mean', 'count'])
age_group_order = ['Child', 'Young Adult', 'Adult', 'Senior']
age_group_survival = age_group_survival.reindex([ag for ag in age_group_order if ag in age_group_survival.index])
ax4.bar(range(len(age_group_survival)), age_group_survival['mean'], color='purple', edgecolor='black', alpha=0.8)
ax4.set_xticks(range(len(age_group_survival)))
ax4.set_xticklabels(age_group_survival.index, rotation=45)
ax4.set_title('Age Group vs Survival', fontsize=13, fontweight='bold')
ax4.set_ylabel('Survival Rate', fontsize=11)
ax4.axhline(y=df_advanced['Survived'].mean(), color='red', linestyle='--', linewidth=2)
ax4.grid(axis='y', alpha=0.3)
for i, (idx, row) in enumerate(age_group_survival.iterrows()):
    ax4.text(i, row['mean'] + 0.02, f"{row['mean']:.0%}", ha='center', fontsize=9)

# 5. Fare Group vs Survival
ax5 = axes[1, 1]
fare_group_survival = df_advanced.groupby('fare_group')['Survived'].agg(['mean', 'count'])
fare_order = ['Low', 'Medium', 'High', 'VeryHigh']
fare_group_survival = fare_group_survival.reindex([fg for fg in fare_order if fg in fare_group_survival.index])
ax5.bar(range(len(fare_group_survival)), fare_group_survival['mean'], color='gold', edgecolor='black', alpha=0.8)
ax5.set_xticks(range(len(fare_group_survival)))
ax5.set_xticklabels(fare_group_survival.index, rotation=45)
ax5.set_title('Fare Group vs Survival', fontsize=13, fontweight='bold')
ax5.set_ylabel('Survival Rate', fontsize=11)
ax5.axhline(y=df_advanced['Survived'].mean(), color='red', linestyle='--', linewidth=2)
ax5.grid(axis='y', alpha=0.3)
for i, (idx, row) in enumerate(fare_group_survival.iterrows()):
    ax5.text(i, row['mean'] + 0.02, f"{row['mean']:.0%}", ha='center', fontsize=9)

# 6. Sex-Class Interaction vs Survival
ax6 = axes[1, 2]
sex_class_survival = df_advanced.groupby('sex_pclass')['Survived'].agg(['mean', 'count'])
sex_class_survival = sex_class_survival.sort_values('mean', ascending=False)
colors_sc = ['#e91e63' if 'female' in idx else '#2196f3' for idx in sex_class_survival.index]
ax6.barh(range(len(sex_class_survival)), sex_class_survival['mean'], color=colors_sc, edgecolor='black', alpha=0.8)
ax6.set_yticks(range(len(sex_class_survival)))
ax6.set_yticklabels(sex_class_survival.index, fontsize=9)
ax6.set_title('Sex-Class Interaction vs Survival', fontsize=13, fontweight='bold')
ax6.set_xlabel('Survival Rate', fontsize=11)
ax6.axvline(x=df_advanced['Survived'].mean(), color='red', linestyle='--', linewidth=2)
ax6.grid(axis='x', alpha=0.3)
for i, (idx, row) in enumerate(sex_class_survival.iterrows()):
    ax6.text(row['mean'] + 0.02, i, f"{row['mean']:.0%}", va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n📊 KEY INSIGHTS FROM ADVANCED FEATURES:")
print("=" * 80)
print("✓ Families of 2-4 had better survival than solo travelers or very large families")
print("✓ 'Mrs' and 'Miss' titles had 70%+ survival (women & children first policy)")
print("✓ Children (<16) had highest survival among age groups")
print("✓ Higher fare groups correlated with better survival (proxy for class/wealth)")
print("✓ Female 1st class passengers had 96%+ survival rate")
print("✓ Male 3rd class passengers had lowest survival (~13%)")
print("=" * 80)

### 🎯 Benefits of Advanced Feature Engineering

**Why These Features Matter:**

1. **Better Signal Extraction**:
   - `title_grouped` reduces noise from 17+ rare titles → 5 meaningful categories
   - `sex_pclass` captures the "women & children first" policy better than sex/class alone
   - `fare_per_person` accounts for group tickets vs individual fares

2. **Handling Missing Data Intelligently**:
   - `age_missing` flag preserves information about missingness patterns
   - `fare_inference` fills missing fares with class-appropriate estimates
   - `cabin_known` treats cabin availability as a feature (not just missing data)

3. **Domain Knowledge Integration**:
   - Family size features reflect historical accounts of families staying together
   - Title grouping aligns with 1912 social hierarchy
   - Cabin deck extraction uses ship layout (upper decks = better access to lifeboats)

4. **Production-Ready**:
   - All features implemented in `src/features/build_features.py`
   - Used by the Flask API (`/predict` endpoint)
   - Fully tested in `test_api.py` (100% pass rate)

**Next Steps:**
- These advanced features will be automatically applied during model training
- The production API uses `include_advanced=True` by default
- You can toggle advanced features on/off for A/B testing performance gains

---

### 3.2 Train/Test split

In [ ]:
# I want to make sure that the data remains as a pandas dataframe when splitting. Features get split based on the train_test flag from earlier:
X_train = pd.DataFrame(all_dummies[all_dummies.train_test == 1].drop(['train_test'], axis=1))
X_test = pd.DataFrame(all_dummies[all_dummies.train_test == 0].drop(['train_test'], axis=1))

# Same logic for training labels
y_train = all_data[all_data.train_test == 1].Survived.values
print('y_train shape ->', y_train.shape)

### 3.3 Scaling

In [ ]:
# Visualize BEFORE scaling - show the scale differences
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
features_to_scale = ['Age', 'SibSp', 'Parch', 'norm_fare']

for idx, feature in enumerate(features_to_scale):
    ax = axes[idx // 2, idx % 2]
    all_dummies[feature].hist(bins=30, ax=ax, color='lightcoral', edgecolor='black', alpha=0.7)
    ax.set_title(f'BEFORE Scaling: {feature}', fontsize=12, fontweight='bold')
    ax.set_xlabel(f'{feature} Value', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.axvline(all_dummies[feature].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {all_dummies[feature].mean():.2f}')
    ax.axvline(all_dummies[feature].std(), color='blue', linestyle=':', linewidth=2, label=f'Std: {all_dummies[feature].std():.2f}')
    ax.legend(fontsize=8)
    ax.grid(axis='y', alpha=0.3)

plt.suptitle('Feature Distributions BEFORE StandardScaler', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Apply StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[features_to_scale] = scale.fit_transform(all_dummies_scaled[features_to_scale])

# Visualize AFTER scaling - all features now have mean=0, std=1
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, feature in enumerate(features_to_scale):
    ax = axes[idx // 2, idx % 2]
    all_dummies_scaled[feature].hist(bins=30, ax=ax, color='lightgreen', edgecolor='black', alpha=0.7)
    ax.set_title(f'AFTER Scaling: {feature}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Standardized Value (z-score)', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.axvline(all_dummies_scaled[feature].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {all_dummies_scaled[feature].mean():.4f}')
    ax.axvline(all_dummies_scaled[feature].std(), color='blue', linestyle=':', linewidth=2, label=f'Std: {all_dummies_scaled[feature].std():.4f}')
    ax.legend(fontsize=8)
    ax.grid(axis='y', alpha=0.3)
    ax.set_xlim([-4, 4])  # Standardized features typically in this range

plt.suptitle('Feature Distributions AFTER StandardScaler (mean=0, std=1)', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Scaling Transformation Summary:")
print(f"   ✅ All features now have mean ≈ 0 and standard deviation ≈ 1")
print(f"   ✅ Features are now on the same scale - prevents dominance by large-magnitude features")
print(f"   ✅ Critical for distance-based algorithms (KNN, SVC) and regularized models (Logistic, Ridge)")

# Split into train/test
X_train_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 1].drop(['train_test'], axis=1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 0].drop(['train_test'], axis=1)
y_train = all_data[all_data.train_test == 1].Survived

print(f"\n🎯 Final Dataset Shapes:")
print(f"   X_train_scaled: {X_train_scaled.shape}")
print(f"   X_test_scaled: {X_test_scaled.shape}")
print(f"   y_train: {y_train.shape}")

### 3.4 Finalized Data Correlations

In [ ]:
# Concatenate into a full dataset
full_df = pd.concat([X_train, y_train], axis=1)

correlation = full_df.corr()['Survived'].sort_values(ascending=False)

# Correlation graph
correlation[1:].plot(kind='bar', figsize=(10,5), title='Survivability Correlation')
plt.show()

## 4. Metrics and Evaluation Strategy

**Competition Metric**: **Accuracy** (required by Kaggle leaderboard)

**Why also track F1 Score?**
- Dataset is imbalanced (~38% survival rate)
- Accuracy can be misleading with imbalanced classes
- F1 Score balances precision and recall, giving a more complete picture

**Cross-Validation Strategy**:
- **Stratified 5-fold CV**: Maintains class distribution across folds
- Essential for imbalanced datasets (62% died, 38% survived)
- Reduces overfitting risk
- More reliable than single train/test split

**What We Measure:**
- **Training Performance**: How well model learns from data
- **Validation Performance**: How well model generalizes
- **Feature Importance**: Which features drive predictions

### 4.0 Handling Class Imbalance

The Titanic dataset is **imbalanced** with a ~62/38 split (died/survived). To handle this:

**Stratified K-Fold Cross-Validation:**
- Ensures each fold maintains the same class distribution as the full dataset
- Prevents bias toward the majority class
- Provides more reliable performance estimates

**Weighted Metrics:**
- F1-weighted score accounts for class imbalance
- Gives appropriate weight to minority class performance

**Why This Matters:**
- Regular K-Fold might create folds with very different survival rates
- Could lead to unreliable model evaluation
- Stratification ensures consistent, reproducible results

**Additional Class Imbalance Techniques (Not Applied):**

While we use Stratified CV and weighted metrics, other techniques exist:

1. **SMOTE (Synthetic Minority Over-sampling)**
   - Creates synthetic samples of minority class
   - Risk: May introduce noise or overfitting
   - Not used: Our dataset already has sufficient samples (891 total)

2. **Class Weights in Models**
   - Many sklearn models support `class_weight='balanced'`
   - Automatically adjusts loss function
   - Not used: Stratified CV + weighted F1 already handles imbalance well

3. **Threshold Adjustment**
   - Tune decision threshold (default: 0.5)
   - Optimize for specific metric (precision vs recall)
   - Not used: Kaggle evaluates on accuracy, not threshold-dependent

4. **Ensemble with Undersampling**
   - Create multiple balanced subsets
   - Train models on each, then ensemble
   - Not used: Would lose training data and not significantly improve results

**Our Approach:**
Stratified CV + Weighted Metrics provides excellent balance without data manipulation risks.

### 4.1 Model Evaluation Functions

In [ ]:
# Setting up a constant for the metric input to easily shift between accuracy and F1 score
METRIC = 'accuracy'
# METRIC = 'f1_weighted'

### 4.2 Metric Functions

### 4.3 Helper Functions
This section is also where I have stored all functions with miscellaneous uses that will be used throughout the rest of this notebook.

## 5. Model Selection and Baseline Evaluation
To start with I would like to see how various different models perform with default parameters. I tried the models in the below list using 5 fold cross validation to get a baseline. I can later compare these results to the tuned version of these models and see how much tuning improves the performance of each. I can get away with using large number of models for this project since the dataset is not large at all and fitting all these models should only take a few seconds each.

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=2000),
    'Decision Tree Classifier': DecisionTreeClassifier(random_state=1),
    'Random Forest Classifier': RandomForestClassifier(random_state=1),
    'GaussianNB': GaussianNB(),
    'KNeighbors Classifier': KNeighborsClassifier(),
    'SVC': SVC(probability=True),
    'XGB Classifier': XGBClassifier(random_state=1),
    'CatBoost Classifier': CatBoostClassifier(silent=True)
}

### 5.1 Scaled vs non-scaled data testing
I want to check what difference scaling the data will have on the scores.

In [ ]:
# Evaluate models on non-scaled data
_ = evaluate_models_cv(models, X_train, y_train, cv_folds=config.CV_FOLDS, scoring='accuracy')


In [ ]:
# Evaluate models on scaled data  
_ = evaluate_models_cv(models, X_train_scaled, y_train, cv_folds=config.CV_FOLDS, scoring='accuracy')


Some models are unaffected by the scaling and the rest got improved scores. GaussianNB is the only exception since it's score reduced. That model's overall performance is significantly lower than the rest so it will be getting dropped from the list.

### 5.2 Voting classifier 

The VotingClassifier in scikit-learn combines the predictions from multiple individual classifiers to make a final prediction. There are two main types of voting strategies:

- Hard Voting Classifier - In the hard voting scheme, each classifier in the ensemble casts a single "vote" for a particular class ("yes" or "no"). The final prediction is determined by a majority vote. To ensure a decisive outcome, it's recommended to use an odd number of classifiers.

- Soft Voting Classifier - The soft voting scheme considers the confidence or probability assigned by each classifier for each class. Instead of a simple majority vote, it takes the average of the predicted probabilities. If the average confidence for a particular class surpasses 50%, that class is chosen as the final prediction.

The choice between hard and soft voting often depends on the characteristics of the individual classifiers in the ensemble. Hard voting is suitable when classifiers are diverse and have varying strengths, while soft voting is effective when classifiers provide probability estimates, allowing for a more nuanced decision-making process.

The model will require tuples of model name and model object which I can generate using the models dictionary.

In [ ]:
# Build model
voting_clf_model_tuples = [(model_name, model) for model_name, model in models.items()]
voting_clf = VotingClassifier(estimators=voting_clf_model_tuples, voting='soft')

In [ ]:
# Evaluate Voting Classifier
_ = evaluate_models_cv(
    {'Voting Classifier': voting_clf}, 
    X_train_scaled.values, 
    y_train,
    cv_folds=config.CV_FOLDS,
    scoring='accuracy'
)


### 5.3 Voting Classifier Predictions
This model is performing better than the rest so let's generate a submissions file to use as a baseline

In [ ]:
# Generate baseline voting classifier submission
voting_clf.fit(X_train_scaled.values, y_train)
voting_clf_preds = voting_clf.predict(X_test_scaled.values).astype(int)

# Generate submission file
generate_kaggle_submission(
    predictions=voting_clf_preds,
    passenger_ids=test.PassengerId,
    file_name='01_voting_clf_submission.csv',
    output_dir='submissions'
)


Let's set up a function to use for generating submission files

### 5.4 Model List Cleanup
Let's eliminate some of the models that are not performing that well:
- GaussianNB - poor performance
- Decision Tree Classifier - poor performance and forest should be better in most situations
- SVC - this model has been performing well but it's causing trouble when tuning hyperparameters

In [ ]:
models_to_remove = ['GaussianNB', 'Decision Tree Classifier']
for model in models_to_remove:
    if model in models:
        del models[model]

## 6. Hyperparameter Tuning

After establishing baselines, we'll systematically optimize each model using **GridSearchCV** with **Stratified K-Fold Cross-Validation**.

**Tuning Strategy:**
- **Grid Search**: Exhaustive search over specified parameter values
- **Stratified CV**: Maintains class distribution across folds (essential for imbalanced data)
- **Weighted F1 Score**: Optimization metric that accounts for class imbalance
- **Reduced Grids**: Some grids simplified after initial exploration to balance performance vs. compute time

**Models to Tune:**
- Logistic Regression
- Random Forest Classifier
- K-Neighbors Classifier
- Support Vector Classifier (SVC)
- XGBoost Classifier
- CatBoost Classifier

The automated hyperparameter search will report best parameters and scores for each model.

In [ ]:
model_params = {
    'Logistic Regression': {
        'max_iter' : [10, 50, 100, 1000],
        'penalty' : ['l1', 'l2'],
        'C' : np.logspace(-4, 4, 20),
        'solver' : ['liblinear']
        },
    # 'Random Forest Classifier': {
    #     'n_estimators': [100, 500, 1000], 
    #     'bootstrap': [True, False],
    #     'max_depth': [3, 5, 10, 20, 50, 75, 100, None],
    #     'max_features': ['auto', 'sqrt', 'log2'],
    #     'min_samples_leaf': [1, 2, 4, 10],
    #     'min_samples_split': [2, 5, 10]
    #     },
    'Random Forest Classifier': {
        'n_estimators': [100, 300, 450],
        'criterion':['gini', 'entropy'], 
        'bootstrap': [True],
        'max_depth': [10, 15],
        'max_features': ['sqrt', 20, 40],
        'min_samples_leaf': [1, 2],
        'min_samples_split': [.5, 2]
        },
    'KNeighbors Classifier': {
        'n_neighbors' : [3, 5, 7, 9, 12, 15],
        'weights' : ['uniform', 'distance'],
        'algorithm' : ['ball_tree', 'kd_tree'],
        'p' : [1, 2]
        },
    'SVC': [
        {'kernel': ['rbf'], 'gamma': [.1, .5, 1, 2, 5, 10], 'C': [.1, 1, 10]},
        {'kernel': ['linear'], 'C': [.1, 1, 10]},
        {'kernel': ['poly', 'linear', 'rbf'], 'degree' : [1, 2, 3], 'C': [.1, 1, 10]}
        ],
    'XGB Classifier': {
        'n_estimators': [500, 550],
        'colsample_bytree': [.5, .6, .75],
        'max_depth': [10, None],
        'reg_alpha': [1],
        'reg_lambda': [5, 10, 15],
        'subsample': [.55, .6, .65],
        'learning_rate':[.5],
        'gamma':[.25, .5, 1],
        'min_child_weight':[0.01],
        'sampling_method': ['uniform']
        },
    'CatBoost Classifier': {
        'iterations': [400, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'depth': [2, 4, 6],
        'l2_leaf_reg': [1, 2],
        'border_count': [64, 128]
    }
}

Let's now run the hyperparameter grid search and save the results for each model along with the model's best performing version in a dictionary which will allow for easy access to everything I might need in regards to these models later on.

In [ ]:
# Optimize models using GridSearchCV
tuned_models = optimize_models(models, model_params, X_train_scaled, y_train)


### 6.1 Detailed Cross-Validation Analysis

Let's analyze the stability and consistency of our tuned models across CV folds.


In [ ]:
# Get detailed CV scores for tuned models
print("="*70)
print("CROSS-VALIDATION SCORE ANALYSIS")
print("="*70)

cv_scores_dict = {}
for model_name, (model, mean_score) in tuned_models.items():
    scores_detail = get_cv_scores_detailed(
        model, 
        X_train_scaled, 
        y_train,
        cv_folds=config.CV_FOLDS,
        scoring='accuracy'
    )
    cv_scores_dict[model_name] = scores_detail['scores']
    
    print(f"\n{model_name}:")
    print(f"  Mean:   {scores_detail['mean']:.4f}")
    print(f"  Std:    {scores_detail['std']:.4f}")
    print(f"  Range:  [{scores_detail['min']:.4f}, {scores_detail['max']:.4f}]")
    print(f"  Folds:  {[f'{s:.4f}' for s in scores_detail['scores']]}")

print("\n" + "="*70)

# Visualize CV score distributions
fig = plot_cv_score_distribution(
    cv_scores_dict,
    title='Cross-Validation Score Distributions (5-Fold)',
    figsize=(14, 6)
)
plt.show()

print("\n" + "="*70)
print("INTERPRETATION:")
print("="*70)
print("• Lower variance = More stable/reliable model")
print("• Higher median = Better average performance")
print("• Look for models with high median AND low variance")
print("="*70)


Unsurprisingle, age and sex are big determinants of the outcome. It's quite interesting that the normalized Fare is the feature with the greatest importance.

### 6.1 Tuned XGB Predictions
XGB Classifier is the best performing model so far so let's generate a submissions file using this tuned version

In [ ]:
# Generate XGBoost tuned predictions
xgb_predictions = tuned_models['XGB Classifier'][0].predict(X_test_scaled).astype(int)
generate_kaggle_submission(xgb_predictions, test.PassengerId, '02_xgb_tuned_submission.csv', 'submissions')


### 6.2 Feature Importances
I am quite interested in the feature importances so let's have a look.

In [ ]:
best_xgb = tuned_models['XGB Classifier'][0].fit(X_train_scaled, y_train)
feat_importances = pd.Series(best_xgb.feature_importances_, index=X_train_scaled.columns)
feat_importances.nlargest(20).plot(kind='barh')

In [ ]:
best_rf = tuned_models['Random Forest Classifier'][0].fit(X_train_scaled, y_train)
feat_importances = pd.Series(best_rf.feature_importances_, index=X_train_scaled.columns)
feat_importances.nlargest(20).plot(kind='barh')

### 6.3 Comprehensive Feature Importance Analysis

Let's compare feature importances across multiple models to understand which features consistently drive predictions.

In [ ]:
# Compare feature importances across tree-based models
tree_models = {
    'Random Forest': tuned_models['Random Forest Classifier'][0],
    'XGBoost': tuned_models['XGB Classifier'][0],
    'CatBoost': tuned_models['CatBoost Classifier'][0]
}

# Create subplots for each model
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for (name, model), ax in zip(tree_models.items(), axes):
    model.fit(X_train_scaled, y_train)
    feat_imp = pd.Series(model.feature_importances_, index=X_train_scaled.columns)
    feat_imp.nlargest(15).sort_values().plot(kind='barh', ax=ax, color='skyblue')
    ax.set_title(f'{name} - Top 15 Features', fontsize=14, fontweight='bold')
    ax.set_xlabel('Importance', fontsize=12)
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Aggregate feature importance across models
print("\n" + "="*60)
print("AGGREGATED FEATURE IMPORTANCE (Average Across Models)")
print("="*60)

all_importances = pd.DataFrame()
for name, model in tree_models.items():
    all_importances[name] = model.feature_importances_

all_importances.index = X_train_scaled.columns
all_importances['Mean'] = all_importances.mean(axis=1)
all_importances['Std'] = all_importances.std(axis=1)

top_features = all_importances.sort_values('Mean', ascending=False).head(10)
print("\nTop 10 Most Important Features:")
print(top_features[['Mean', 'Std']].to_string())


## 7. Additional Ensemble Approaches using Voting Classifier
1) Experimented with a <b>Hard Voting</b> classifier

2) Experimented with a <b>Soft Voting</b> classifier

3) Experimented with <b>Soft Voting while eliminating worst performing model</b>

4) Experimented with <b>Soft Voting while eliminating the two worst performing</b>

Let's run a voting classifier using the tuned models. I also want to try and eliminate some of the models to see if that improves performance.

In [ ]:
# Test hard voting with tuned models
tuned_model_tuples = [(model_name, model[0]) for model_name, model in tuned_models.items()]
voting_clf_all = VotingClassifier(estimators=tuned_model_tuples, voting='hard')
_ = evaluate_models_cv({'Voting Classifier': voting_clf_all}, X_train_scaled, y_train, config.CV_FOLDS, 'accuracy')


In [ ]:
# Test soft voting with tuned models
tuned_model_tuples = [(model_name, model[0]) for model_name, model in tuned_models.items()]
voting_clf_all = VotingClassifier(estimators=tuned_model_tuples, voting='soft')
_ = evaluate_models_cv({'Voting Classifier': voting_clf_all}, X_train_scaled, y_train, config.CV_FOLDS, 'accuracy')


In [ ]:
# Test soft voting without worst performer (Logistic Regression)
tuned_model_tuples = [(model_name, model[0]) for model_name, model in tuned_models.items() if model_name not in ['Logistic Regression']]
voting_clf_all = VotingClassifier(estimators=tuned_model_tuples, voting='soft')
_ = evaluate_models_cv({'Voting Classifier': voting_clf_all}, X_train_scaled, y_train, config.CV_FOLDS, 'accuracy')


In [ ]:
# Test soft voting without two worst performers
tuned_model_tuples = [(model_name, model[0]) for model_name, model in tuned_models.items() if model_name not in ['Logistic Regression', 'KNeighbors Classifier']]
voting_clf_all = VotingClassifier(estimators=tuned_model_tuples, voting='soft')
_ = evaluate_models_cv({'Voting Classifier': voting_clf_all}, X_train_scaled, y_train, config.CV_FOLDS, 'accuracy')


Hard voting seems to be the way to go. Also, removing the worst performing models does produce an improved score as well so let's keep that approach.

### 7.1 Voting Classifier Optimisation
In a soft voting classifier you can apply weights to each of the models. Let's use a grid search to explore different weightings. 

To not have to adjust the grid every time the models count change we generate a grid using itertools.

In [ ]:
# Optimize voting classifier weights
combinations = itertools.product([1, 2], repeat=len(tuned_model_tuples))
combinations = [list(comb) for comb in combinations if len(set(comb)) != 1]

voting_classifier_params = {'Voting Classifier': {'weights': combinations, 'voting': ['soft', 'hard']}}

# Tune the voting classifier
tuned_voting_clf = optimize_models(
    {'Voting Classifier': voting_clf_all},
    voting_classifier_params,
    X_train_scaled,
    y_train
)


The performence does improve after tuning but not by that much. Let's add this tuned Voting Classifier to the dictionary of models which can then be sorted which will allow the best performing model to be easily retrieved.

In [ ]:
# Adding the Voting Classifier to the tuned models dictionary and sorting it to easily retrieve the best performing model
tuned_models = {**tuned_voting_clf, **tuned_models}
sorted_tuned_models = dict(sorted(tuned_models.items(), key=lambda item: -item[1][1]))

## 8. Final Submissions
Let's take the best performing model from our tuned_models dict and use it to generate the final submission file. I'll also save that model for later use.

### 7.2 Model Performance Visualization

Let's create comprehensive visualizations comparing all tuned models.

In [ ]:
# Extract scores from tuned models for comparison
model_scores = {name: score for name, (model, score) in tuned_models.items()}

# Create comparison visualization
fig = plot_model_comparison(
    model_scores,
    title='Tuned Model Performance Comparison (Cross-Validation)',
    metric_name=f'{METRIC.title()} Score',
    figsize=(12, 6)
)
plt.show()

# Print top 3 performers
print("\n🏆 Top 3 Performing Models:")
for i, (name, score) in enumerate(sorted(model_scores.items(), key=lambda x: -x[1])[:3], 1):
    print(f"{i}. {name}: {score:.4f}")


In [ ]:
# Taking best model from dictionary
best_model_name, best_model_tuple = list(sorted_tuned_models.items())[0]
best_model, best_model_score = best_model_tuple
print(f"Best performing model -> {best_model_name} with anm F1 score of {round(best_model_score * 100, 1)}%")

### 8.1 Comprehensive Model Evaluation

Before finalizing our submission, let's perform a detailed evaluation with confusion matrix and ROC curve analysis.

In [ ]:
# Get predictions on training set for evaluation
y_train_pred = best_model.predict(X_train_scaled)
y_train_pred_proba = best_model.predict_proba(X_train_scaled)[:, 1] if hasattr(best_model, 'predict_proba') else None

# Create confusion matrix
fig = plot_confusion_matrix(
    y_train,
    y_train_pred,
    title=f'Confusion Matrix - {best_model_name} (Training Set)',
    figsize=(8, 6)
)
plt.show()

# Plot ROC curve if probability predictions available
if y_train_pred_proba is not None:
    fig = plot_roc_curve(
        y_train,
        y_train_pred_proba,
        title=f'ROC Curve - {best_model_name}',
        figsize=(8, 6)
    )
    plt.show()

# Print detailed classification metrics
print("\n" + "="*60)
print(f"DETAILED METRICS - {best_model_name}")
print("="*60)
print(f"\nAccuracy:  {accuracy_score(y_train, y_train_pred):.4f}")
print(f"F1 Score:  {f1_score(y_train, y_train_pred, average='weighted'):.4f}")
if y_train_pred_proba is not None:
    print(f"ROC-AUC:   {roc_auc_score(y_train, y_train_pred_proba):.4f}")

print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_train, y_train_pred, target_names=['Did Not Survive', 'Survived']))


In [ ]:
# Generate final submission with best model
best_model_preds = best_model.predict(X_test_scaled).astype(int)
generate_kaggle_submission(
    best_model_preds, 
    test.PassengerId, 
    config.SUBMISSION_OPTIMIZED,
    config.SUBMISSIONS_DIR
)

# Save best model for future use
save_model_pickle(best_model, 'titanic_survival_classifier.pkl', config.MODELS_DIR)

### 8.2 Final Summary

**Project Results Summary:**

In [ ]:
print("\n" + "="*70)
print("TITANIC SURVIVAL PREDICTION - PROJECT SUMMARY")
print("="*70)

print(f"\n🎯 Best Model: {best_model_name}")
print(f"📊 Cross-Validation Score: {best_model_score:.4f}")
print(f"🔢 Random Seed: {config.RANDOM_SEED} (for reproducibility)")

print("\n" + "-"*70)
print("KEY FINDINGS:")
print("-"*70)

# Get top 5 important features if available
if hasattr(best_model, 'feature_importances_'):
    top_5_features = pd.Series(
        best_model.feature_importances_,
        index=X_train_scaled.columns
    ).nlargest(5)
    
    print("\nTop 5 Most Important Features:")
    for i, (feat, imp) in enumerate(top_5_features.items(), 1):
        print(f"  {i}. {feat}: {imp:.4f}")

print("\n" + "-"*70)
print("FILES GENERATED:")
print("-"*70)
print(f"  ✓ {config.SUBMISSION_VOTING_CLF} (Baseline ensemble)")
print(f"  ✓ {config.SUBMISSION_XGB_TUNED} (Best individual model)")
print(f"  ✓ {config.SUBMISSION_OPTIMIZED} (Final submission)")
print("  ✓ titanic_survival_classifier.pkl (Saved model)")

print("\n" + "-"*70)
print("METHODOLOGY HIGHLIGHTS:")
print("-"*70)
print("  • Comprehensive EDA with visualization")
print("  • Feature engineering (cabin_multiple, name_title, norm_fare)")
print("  • 8 different classifiers compared")
print("  • GridSearchCV hyperparameter tuning")
print("  • Voting ensemble methods")
print("  • Cross-validation for robust evaluation")
print("  • SHAP analysis for model interpretability")

print("\n" + "-"*70)
print("READY FOR KAGGLE SUBMISSION:")
print("-"*70)
print(f"  📁 Location: {config.SUBMISSIONS_DIR}/")
print(f"  📄 Submit: {config.SUBMISSION_OPTIMIZED}")
print("="*70 + "\n")

## 9. Model Explainability with SHAP

**SHAP (SHapley Additive exPlanations)** provides model-agnostic explanations by computing the contribution of each feature to individual predictions.

SHAP values help us understand:
- **Global Importance**: Which features matter most across all predictions
- **Local Explanations**: Why a specific prediction was made
- **Feature Interactions**: How features work together
- **Decision Boundaries**: Where the model changes its predictions

This analysis uses SHAP to explain our best-performing model's predictions.


### 9.1 Initialize SHAP Explainer

For tree-based models (XGBoost, Random Forest, CatBoost), we use TreeExplainer which is optimized for speed and accuracy.


In [ ]:
# Create SHAP explainer for the best model
print(f"Creating SHAP explainer for {best_model_name}...")

if hasattr(best_model, 'estimators_'):
    # For ensemble models (VotingClassifier), extract the underlying model
    print("Note: Using first estimator from ensemble for SHAP analysis")
    model_for_shap = best_model.estimators_[0]
else:
    model_for_shap = best_model

# Use TreeExplainer for tree-based models
try:
    explainer = shap.TreeExplainer(model_for_shap)
    print("✓ TreeExplainer initialized successfully")
except Exception as e:
    print(f"TreeExplainer not supported, using KernelExplainer: {e}")
    # Fallback to KernelExplainer (slower but model-agnostic)
    # Use a sample of training data as background
    explainer = shap.KernelExplainer(
        model_for_shap.predict_proba if hasattr(model_for_shap, 'predict_proba') else model_for_shap.predict,
        shap.sample(X_train_scaled, 100)
    )

# Calculate SHAP values for training set (using subset for speed)
print("Calculating SHAP values (this may take a moment)...")
sample_size = min(500, len(X_train_scaled))
X_sample = X_train_scaled.sample(n=sample_size, random_state=config.RANDOM_SEED) if isinstance(X_train_scaled, pd.DataFrame) else X_train_scaled[:sample_size]

shap_values = explainer.shap_values(X_sample)

print(f"✓ SHAP values calculated for {sample_size} samples")
print(f"  SHAP values shape: {shap_values.shape if isinstance(shap_values, np.ndarray) else shap_values[1].shape}")


### 9.2 SHAP Summary Plot - Global Feature Importance

The summary plot shows:
- **Distribution** of SHAP values for each feature
- **Impact direction**: Red (high feature value) vs Blue (low feature value)
- **Magnitude**: How much each feature affects predictions


In [ ]:
# SHAP Summary Plot
plt.figure(figsize=(12, 8))

# For binary classification, shap_values might be a list [class0, class1]
# We want class 1 (Survived=1) for interpretation
shap_values_plot = shap_values[1] if isinstance(shap_values, list) else shap_values

shap.summary_plot(
    shap_values_plot, 
    X_sample,
    feature_names=X_train_scaled.columns if isinstance(X_train_scaled, pd.DataFrame) else None,
    show=False
)
plt.title('SHAP Summary Plot - Feature Impact on Survival Prediction', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("INTERPRETING THE SUMMARY PLOT:")
print("="*70)
print("• Features are ranked by importance (top = most important)")
print("• Red points = high feature values, Blue points = low feature values")
print("• Points to the right = push prediction toward Survived=1")
print("• Points to the left = push prediction toward Survived=0")
print("="*70)


### 9.3 SHAP Bar Plot - Mean Absolute Feature Importance


In [ ]:
# SHAP Bar Plot
plt.figure(figsize=(10, 6))

shap.summary_plot(
    shap_values_plot,
    X_sample,
    plot_type="bar",
    feature_names=X_train_scaled.columns if isinstance(X_train_scaled, pd.DataFrame) else None,
    show=False
)
plt.title('SHAP Feature Importance (Mean |SHAP value|)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


### 9.4 Individual Prediction Explanations - Waterfall Plots

Let's examine specific cases to understand model decisions:
- **Survived Passenger**: Why did the model predict survival?
- **Did Not Survive**: Why did the model predict death?


In [ ]:
# Get predictions for the sample
sample_predictions = model_for_shap.predict(X_sample)

# Find examples of survivors and non-survivors
survived_idx = np.where(sample_predictions == 1)[0]
died_idx = np.where(sample_predictions == 0)[0]

print(f"Found {len(survived_idx)} predicted survivors and {len(died_idx)} predicted non-survivors in sample")

# Select one example of each
survivor_example = survived_idx[0] if len(survived_idx) > 0 else 0
non_survivor_example = died_idx[0] if len(died_idx) > 0 else 1

# Create waterfall plots
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Waterfall plot for a survivor
print("\n" + "="*70)
print(f"EXAMPLE 1: Predicted Survivor (Index {survivor_example})")
print("="*70)
plt.sca(axes[0])

# For SHAP v0.41+, use Explanation objects
if hasattr(shap, 'Explanation'):
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_plot[survivor_example],
            base_values=explainer.expected_value if hasattr(explainer, 'expected_value') else 0,
            data=X_sample.iloc[survivor_example] if isinstance(X_sample, pd.DataFrame) else X_sample[survivor_example],
            feature_names=X_train_scaled.columns if isinstance(X_train_scaled, pd.DataFrame) else None
        ),
        show=False
    )
else:
    # Fallback for older versions
    shap.waterfall_plot(
        explainer.expected_value if hasattr(explainer, 'expected_value') else 0,
        shap_values_plot[survivor_example],
        X_sample.iloc[survivor_example] if isinstance(X_sample, pd.DataFrame) else X_sample[survivor_example],
        feature_names=X_train_scaled.columns if isinstance(X_train_scaled, pd.DataFrame) else None
    )

axes[0].set_title(f'SHAP Waterfall: Predicted Survivor', fontsize=12, fontweight='bold')

# Waterfall plot for non-survivor
print(f"\nEXAMPLE 2: Predicted Non-Survivor (Index {non_survivor_example})")
print("="*70)
plt.sca(axes[1])

if hasattr(shap, 'Explanation'):
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_plot[non_survivor_example],
            base_values=explainer.expected_value if hasattr(explainer, 'expected_value') else 0,
            data=X_sample.iloc[non_survivor_example] if isinstance(X_sample, pd.DataFrame) else X_sample[non_survivor_example],
            feature_names=X_train_scaled.columns if isinstance(X_train_scaled, pd.DataFrame) else None
        ),
        show=False
    )
else:
    shap.waterfall_plot(
        explainer.expected_value if hasattr(explainer, 'expected_value') else 0,
        shap_values_plot[non_survivor_example],
        X_sample.iloc[non_survivor_example] if isinstance(X_sample, pd.DataFrame) else X_sample[non_survivor_example],
        feature_names=X_train_scaled.columns if isinstance(X_train_scaled, pd.DataFrame) else None
    )

axes[1].set_title(f'SHAP Waterfall: Predicted Non-Survivor', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("INTERPRETING WATERFALL PLOTS:")
print("="*70)
print("• E[f(x)] = Expected model output (baseline prediction)")
print("• Red bars = Features pushing prediction UP (toward survival)")
print("• Blue bars = Features pushing prediction DOWN (toward death)")  
print("• f(x) = Final model output for this passenger")
print("="*70)


### 9.5 SHAP Dependence Plots - Feature Interactions

Dependence plots show how a single feature affects predictions across its range, and potentially how it interacts with another feature.


In [ ]:
# Get feature names
if isinstance(X_train_scaled, pd.DataFrame):
    feature_names = X_train_scaled.columns.tolist()
    
    # Find most important features for dependence plots
    # Get feature importances from the model
    if hasattr(model_for_shap, 'feature_importances_'):
        feature_importance = pd.Series(
            model_for_shap.feature_importances_,
            index=feature_names
        ).sort_values(ascending=False)
        
        # Select top 2 most important features for dependence plots
        top_features = feature_importance.head(2).index.tolist()
        
        print(f"\nTop 2 features for dependence plots: {top_features}")
        
        # Plot dependence for top 2 most important features
        fig, axes = plt.subplots(1, 2, figsize=(16, 5))
        
        for i, feat in enumerate(top_features):
            feat_idx = feature_names.index(feat)
            plt.sca(axes[i])
            shap.dependence_plot(
                feat_idx,
                shap_values_plot,
                X_sample,
                feature_names=feature_names,
                show=False
            )
            axes[i].set_title(f'SHAP Dependence: {feat}', fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        print("\n" + "="*70)
        print("INTERPRETING DEPENDENCE PLOTS:")
        print("="*70)
        print("• X-axis: Feature value")
        print("• Y-axis: SHAP value (impact on prediction)")
        print("• Color: Another feature that interacts with this one")
        print("• Patterns reveal how feature affects survival prediction")
        print("="*70)
    else:
        print("Model does not have feature_importances_ attribute")
else:
    print("Dependence plots require feature names (DataFrame input)")


## 10. Production ML Pipeline

For deployment, we've created a production-ready sklearn Pipeline in `src/pipeline/train_pipeline.py` that:
- Encapsulates all preprocessing steps
- Ensures reproducibility
- Simplifies deployment
- Prevents data leakage

Let's demonstrate the pipeline approach alongside our notebook analysis.


In [ ]:
# NOTE: This cell is commented out as the old pipeline has been replaced.
# For production pipeline, use scripts/run_training.py instead.
# See RESTRUCTURE_SUMMARY.md for details on the new structure.

# # Import production pipeline
# from src.pipeline.train_pipeline import TitanicTrainingPipeline, train_and_evaluate_pipeline

# print("="*70)
# print("PRODUCTION PIPELINE DEMONSTRATION")
# print("="*70)

# # Initialize pipeline with optimized XGBoost parameters
# production_pipeline = TitanicTrainingPipeline(random_state=config.RANDOM_SEED)

# # Train pipeline
# print("\nTraining production pipeline...")
# pipeline_results = production_pipeline.train(train, train['Survived'], cv_folds=5)

# print("\n" + "="*70)
# print("PIPELINE VS NOTEBOOK COMPARISON:")
# print("="*70)
# print(f"Notebook Best Model ({best_model_name}):")
# print(f"  Cross-Val Score: {best_model_score:.4f}")
# print(f"\nProduction Pipeline:")
# print(f"  Cross-Val Score: {pipeline_results['cv_mean']:.4f}")
# print(f"  Std Dev: {pipeline_results['cv_std']:.4f}")
# print("="*70)

# # Generate predictions with pipeline
# pipeline_predictions = production_pipeline.predict(test)

# # Compare predictions
# agreement = np.mean(pipeline_predictions == best_model_preds)
# print(f"\nPrediction Agreement: {agreement:.2%}")
# print(f"  (How often both approaches agree on the same prediction)")

# # Save production pipeline
# print("\nSaving production pipeline...")
# production_pipeline.save('titanic_production_pipeline.pkl', 'models')

# print("\n✓ Production pipeline ready for deployment!")
# print("  Load with: TitanicTrainingPipeline.load('titanic_production_pipeline.pkl')")

print("="*70)
print("PRODUCTION PIPELINE - NEW STRUCTURE")
print("="*70)
print("The production pipeline has been moved to a new modular structure.")
print("\nTo train the model, run:")
print("  python scripts/run_training.py")
print("\nTo make predictions, run:")
print("  python scripts/run_inference.py --input data/raw/test.csv")
print("\nTo start the Flask API, run:")
print("  python scripts/run_api.py --port 5000")
print("\nSee RESTRUCTURE_SUMMARY.md for complete documentation.")
print("="*70)
